In [ ]:
def readTxtFile(inputFile):
    myFile = open(inputFile, "r")
    lines = []
    variables = dict()
    for line in myFile:
        line = line.strip()
        temp = line.split(" ")
        lines.append(temp)

    delx = float(lines[0][0])
    dely = float(lines[0][1])
    dx = float(lines[0][2])
    dy = float(lines[0][3])
    sigmaSquared = float(lines[0][4])
    M = int(lines[1][0])
    N = int(lines[1][1])
    bigImage = []
    for row in lines[2:12]:
        bigImage.append([])
        for pixel in row:
            bigImage[-1].append(float(pixel))
    m = int(lines[12][0])
    n = int(lines[12][1])
    smallImage = []
    for row in lines[13:19]:
        smallImage.append([])
        for pixel in row:
            smallImage[-1].append(float(pixel))
    return(delx,dely,dx,dy,sigmaSquared,M,N,bigImage,m,n,smallImage)

#readTxtFile("PS2_test_1.txt")

# Intensity of the image at a real-valued point (x', y') by the interpolation formula 
# I(A, x', y')
def interpolationFormula(M, N, sigmaSquared, A, x, y):
    numerator = 0
    denominator =0 
    for p in range(M):
        for q in range(N):
            difference = (x-p)**2 + (y-q)**2
            numerator += A[p][q]*math.exp(-sigmaSquared*difference)
            denominator += math.exp(-sigmaSquared*difference)
    intensity = numerator/denominator
    return intensity 

# f(A, T, x, y)
def objectiveFunction(M, N, sigmaSquared, m, n, A, T, x, y):
    optimizationSum = 0
    for i in range(m):
        for j in range(n):
            interpolatedIntensity = interpolationFormula(M, N, sigmaSquared, A, i+x, j+y)
            optimizationSum += (T[i][j] - interpolatedIntensity)**2
    return optimizationSum

# 4a
def gradient(M, N, sigmaSquared, dx, dy, m, n, A, T, x, y):
    secondOrderXSum = 0
    secondOrderYSum = 0
    for i in range(m):
        for j in range(n):
            firstTermX = (T[i][j]-interpolationFormula(M, N, sigmaSquared, A, i+x+dx, j+y))**2
            secondTermX = (T[i][j]-interpolationFormula(M, N, sigmaSquared, A, i+x-dx, j+y))**2
            secondOrderXSum += firstTermX-secondTermX
            firstTermY = (T[i][j]-interpolationFormula(M, N, sigmaSquared, A, i+x, j+y+dy))**2
            secondTermY = (T[i][j]-interpolationFormula(M, N, sigmaSquared, A, i+x, j+y-dy))**2
            secondOrderYSum += firstTermY-secondTermY
    secondOrderX = secondOrderXSum/2*dx
    secondOrderY = secondOrderYSum/2*dy

    return(secondOrderX, secondOrderY)


# 4b
def calculateHessian(M, N, sigmaSquared, dx, dy, m, n, A, T, x, y):
    # doble derivative wrt x 
    num = (objectiveFunction(M, N, sigmaSquared, m, n, A, T, x+dx, y) + 
            objectiveFunction(M, N, sigmaSquared, m, n, A, T, x-dx, y) - 
            2*objectiveFunction(M, N, sigmaSquared, m, n, A, T, x, y)
    )
    den = dx**2
    doubDerWRTx = num/den

    # double derivative wrt y 
    n = (objectiveFunction(M, N, sigmaSquared, m, n, A, T, x, y+dy) + 
        objectiveFunction(M, N, sigmaSquared, m, n, A, T, x, y-dy) -
        2*objectiveFunction(M, N, sigmaSquared, m, n, A, T, x, y)
    )
    d = dy**2 
    doubDerWRTy = n/d

    # double derivative wrt to both x and y 
    numerator = (objectiveFunction(M, N, sigmaSquared, m, n, A, T, x+dx, y+dy) - 
                objectiveFunction(M, N, sigmaSquared, m, n, A, T, x+dx, y-dy) -
                objectiveFunction(M, N, sigmaSquared, m, n, A, T, x-dx, y+dy) + 
                objectiveFunction(M, N, sigmaSquared, m, n, A, T, x-dx, y-dy)
    )
    denominator = 4*dy*dx
    doubleDerWRTxy = numerator/denominator 

    hessian = np.array([doubDerWRTx, doubleDerWRTxy], [doubleDerWRTxy,doubDerWRTy])
    return hessian


def runNewtonRaphson(M, N, sigmaSquared, m, n, A, T, x0, y0, dx, dy):
    min = float('inf')
    for i in range(M-m):
        for j in range(N-n):
            for z in range(10):
                hessianMatrix = calculateHessian(M, N, sigmaSquared, dx, dy, m, n, A, T, x0, y0)
                Gx, Gy = gradient(M, N, sigmaSquared, dx, dy, m, n, A, T, x0, y0)
                G = np.array([Gx, Gy])
                u1 = (hessianMatrix[1][1]*G[0]-hessianMatrix[1][0]*G[1])/(hessianMatrix[0][0]*hessianMatrix[1][1]-(hessianMatrix[0][1]**2))
                u2 = (hessianMatrix[0][0]*G[1]-hessianMatrix[1][0]*G[0])/(hessianMatrix[0][0]*hessianMatrix[1][1]-(hessianMatrix[0][1]**2))
                x0 = x0-u1
                y0 = y0-u2
            newObjective = objectiveFunction(M, N, sigmaSquared, m, n, A, T, x0, y0)
            if newObjective < min:
                point = (x0, y0)
                min = newObjective
    if (M < point[0]) or (0 > point[0]) or (N < point[1]) or (0 > point[1]):
        return(-1,-1)
    else:
        return point  
            
            

def main():
    delx, dely, dx, dy, sigmaSquared, M, N, bigImage, m, n, smallImage = readTxtFile("PS2_test_1.txt")
    NR = runNewtonRaphson(M, N, sigmaSquared, m, n, bigImage, smallImage, 0, 0, dx, dy)
    print(NR)

if __name__ == "__main__":
    main()